# 🤖 MCP Workshop: From Coupled Tools to Decoupled Superpowers

Welcome to this hands-on workshop! You'll learn the difference between:

1. **Traditional AI Agents** - Tools are tightly coupled (hardcoded)
2. **MCP-Powered Agents** - Tools are decoupled and come from external servers

By the end, you'll understand why MCP (Model Context Protocol) is a game-changer for building flexible, powerful AI systems.

---

## 📚 Workshop Structure

| Part | Topic |
|------|-------|
| 1 | Traditional AI Agent with Coupled Tools |
| 2 | The Problem: Agent with No Tools |
| 3 | The Solution: MCP Servers |
| 4 | Build Your Own MCP Server |
| 5 | Connect Agent to MCP Server |
| 6 | Connect to External MCP Servers |

In [ ]:
#First, let's install all dependencies
%%capture
!wget -O requirements.txt -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/requirements.txt
!wget -O mpc_server.py -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/mpc_server.py
!wget -O stdio_mcp_client.py -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/stdio_mcp_client.py
!wget -O lab.py -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/lab.py
# !wget -r files -q https://raw.githubusercontent.com/Sage/talent-arena-2026/master/files/


!python -m ensurepip --upgrade
%pip install --upgrade pip
%pip install -r requirements.txt

In [ ]:
import os

# List of filenames to download
file_list = [
    "customer_summary.txt",
    "incident_log.txt",
    "inventory_alert.txt",
    "product_feedback.txt",
    "q4_report.txt",
    "sales_forecast.txt",
]

# Create the directory if it doesn't exist
os.makedirs("files", exist_ok=True)

# Download each file
base_url = "https://raw.githubusercontent.com/Sage/talent-arena-2026/master/files/"
for filename in file_list:
    url = base_url + filename
    output_path = os.path.join("files", filename)
    !wget -O "$output_path" "$url"

---

## ⚙️ Workshop Setup

Before we begin, you need to configure your connection for the session. 

**📝 Instructions:**
In Colab settings please upload the following:
- `KEY` with the URL provided
- `PASSWORD` with the password provided


In [ ]:
from google.colab import userdata
import os

key = userdata.get('KEY')
password = userdata.get('PASSWORD')

if key:
    os.environ['KEY'] = key
if password:
    os.environ['PASSWORD'] = password

---

# Part 1: Traditional AI Agent with Coupled Tools

Let's start by looking at a **traditional AI agent**. This agent has tools that are **hardcoded** directly into its class - they're tightly coupled and can't be easily changed or shared.

In [ ]:
# First, let's import our lab module
from lab import AIAgent, Tools

print("✅ Lab module imported successfully!")

In [ ]:
# Create an AI Agent with traditional coupled tools
traditional_tools = Tools()
agent = AIAgent(tools=traditional_tools, name="Traditional Agent")

# Let's see what tools this agent has
agent.show_tools()

### 🔍 Notice the Problem?

These database tools are **defined** in the `Tools` class:

```python
class Tools:
    def query_products(self, query):
        # tightly coupled implementation
    def query_sales(self, query):
        # tightly coupled implementation
```

**Problems with coupled tools:**
- 🔒 Tools are locked into this specific agent
- 🔄 Can't easily add/remove tools at runtime
- 🚫 Can't share tools between different agents
- 📦 Every agent needs its own copy of tools

In [ ]:
# Let's try using the agent (it will use its coupled tools)
agent.run("What products are available in the Electronics category?")

In [ ]:
# Now let's ask the agent a question that doesn't require using its tools
agent.run("what is the capital of France?")

---

# Part 2: The Problem - Agent with No Tools

What happens when we **remove all tools** from the agent?

In [ ]:
# Remove all tools from the agent
agent.remove_tools()

# Show the agent's current state
agent.show_tools()

In [ ]:
# Try asking the agent something that requires tools
agent.run("What is the most expensive product we have in our database?")

###  The agent is helpless

Without tools, the agent can only answer from its training data. It **cannot** query our database anymore.

**How do we give this agent superpowers without hardcoding tools?**

---

# Part 3: The Solution - MCP Servers

MCP is an **open protocol** that allows AI agents to connect to **external tool servers**. Instead of hardcoding tools, we:

1. Run tools in a **separate server process**
2. The agent **discovers** available tools dynamically
3. Tools are **decoupled** - they can be shared, updated, or swapped!

```
┌─────────────────┐         ┌─────────────────┐
│    AI Agent     │◄───────►│   MCP Server    │
│  (no hardcoded  │  stdio  │  (provides      │
│   tools)        │         │   tools)        │
└─────────────────┘         └─────────────────┘
```

**Benefits:**
- 🔓 Tools are decoupled from agents
- 🔄 Add/remove tools without changing agent code
- 🤝 Share tools between multiple agents
- 🌐 Connect to tools running anywhere (local or remote)

---

# Part 4: Build Your Own MCP Server! 🛠️

Now it's time to create your own MCP server. Don't worry - we've made it super easy!

You'll use the `MCPServerBuilder` to:
1. Create a server
2. Add pre-built tool packs
3. Start the server

In [ ]:
# First, let's see what tool packs are available
from lab import MCPServerBuilder

MCPServerBuilder.list_available_tool_packs()

In [ ]:
# Step 1: Create your MCP server
my_server = MCPServerBuilder(name="my-workshop-server")

In [ ]:
# Step 2: Add tool packs to your server
# Try adding different combinations!

my_server.add_tool_pack("filesystem")  # File operations
my_server.add_tool_pack("database")    # Database queries

In [ ]:
# Step 3: Review your server configuration
my_server.show_configuration()

In [ ]:
# Step 4: Start your MCP server!
# This launches a subprocess and connects to it
mcp_client = my_server.start()

In [ ]:
# Test your server by calling a tool directly
if mcp_client:
    result = mcp_client.call_tool("query_products", {"category": "Electronics"})
    print("📦 Query Result:")
    print(result)

---

# Part 5: Connect Your Agent to the MCP Server! 🔌

Now for the magic moment - let's connect our **tool-less agent** to the **MCP server**.

The agent will **discover** the tools from the server and become powerful again!

In [ ]:
# Create a tools wrapper that connects to our MCP server
from lab import MCPToolsWrapper

# Wrap the MCP client so the agent can use it
mcp_tools = MCPToolsWrapper(mcp_client)

# Attach the MCP tools to our agent
agent.attach_tools(mcp_tools)

# Rename for clarity
agent.name = "MCP-Powered Agent"

# Show the agent's tools now
agent.show_tools()

In [ ]:
# And access the filesystem
agent.run("Can you list the files in our filesystem?")

### 🎉 Look at that!

The agent now has **new tools** - and they came from the MCP server, not hardcoded!

Let's put it to work...

In [ ]:
# Now the agent can query the database!
agent.run("What Electronics products do we have and which is the most expensive?")

In [ ]:
# It can also analyze data
agent.run("What is our total revenue across all sales?")

In [ ]:
# It can even take actions like generating reports!
agent.run("What files do we currently have in our file system?")

---

# 🏆 THE CHALLENGE: Sales by Region Chart

Now it's your turn! Your goal is to **recreate the expected output chart** by selecting the RIGHT tool packs.

## 📊 The Goal

Create a pie chart showing **Total Sales Value by Region** (EMEA, Americas, APAC).

The expected chart should look like this:
- X-axis: Region names (EMEA, Americas, APAC)
- Y-axis: Total sales value (sum of quantity × price)
- Chart type: Bar chart

## 🎯 The Rules

1. There are **5 tool packs** available, but you can only add **maximum 3**!
2. You must figure out which 3 packs are needed
3. Run the agent and compare your output to the expected chart

## 💡 Hints

- Use `MCPServerBuilder.list_available_tool_packs()` to see available packs
- Some tools require data from other tools to work
- Think about the pipeline: Get data → Transform → Visualize
- **The tools you need might not all be in the same pack!**

## Step 1: View Available Tool Packs

First, let's see what tool packs are available. There are **5 packs** but you can only choose **3**!

In [ ]:
# See all available tool packs and their tools
from lab import MCPServerBuilder

MCPServerBuilder.list_available_tool_packs()

## Step 2: Create Your MCP Server

Create a new server and add ONLY the tool packs you think you need (max 3 out of 5!):

**Remember:** Think about what you need to:
1. Get the sales data from somewhere
2. Prepare/aggregate the data for charting
3. Create the actual chart

⚠️ **Warning:** Adding more than 3 packs will raise an exception!

In [ ]:
# Stop any previous server first
try:
    my_server.stop()
except:
    pass

In [ ]:
# Create your challenge server
challenge_server = MCPServerBuilder(name="challenge-server")

# ADD YOUR TOOL PACKS HERE (max 3!)
#challenge_server.add_tool_pack("filesystem")   # Files: read_file, list_directory, get_file_info
#challenge_server.add_tool_pack("database")     # DB: query_products, query_sales, get_analytics
#challenge_server.add_tool_pack("actions")      # Actions: generate_report, send_notification, create_task
#challenge_server.add_tool_pack("aggregator")   # Data prep: aggregate_for_chart
#challenge_server.add_tool_pack("grapher")      # Charts: create_chart

# Show your configuration
challenge_server.show_configuration()

## Step 3: Start Your Server and Connect Agent

In [ ]:
# Start your server
challenge_client = challenge_server.start()

# Connect agent to your server
from lab import MCPToolsWrapper, AIAgent

if challenge_client:
    challenge_tools = MCPToolsWrapper(challenge_client)
    challenge_agent = AIAgent(tools=challenge_tools, name="Challenge Agent")
    challenge_agent.show_tools()

## Step 4: Run Your Agent!

Ask the agent to create the sales by region chart:

In [ ]:
# Run your agent with the challenge prompt
challenge_agent.run("Create a pie chart showing Total Sales Value by Region.")

## Step 5: View Your Output

Check the `output/` folder for your chart. Compare it to the expected result!

In [ ]:
from IPython.display import Image, display
import os

def show_chart(output_dir="output", image_name=None):
    print("Current working directory:", os.getcwd())
    if not os.path.exists(output_dir):
        print(f"❌ Directory '{output_dir}' not found!")
        return
    all_files = os.listdir(output_dir)
    print(f"Files in '{output_dir}':", all_files)
    chart_files = [f for f in all_files if f.endswith(".png")]
    if not chart_files:
        print("❌ No chart found! Did you add the right tool packs?")
        return
    if image_name and image_name in chart_files:
        chart_to_show = image_name
    else:
        # Sort files by creation time (newest last)
        chart_files_sorted = sorted(chart_files, key=lambda f: os.path.getctime(os.path.join(output_dir, f)))
        chart_to_show = chart_files_sorted[-1]
    print(f"📊 Displaying chart: {chart_to_show}")
    try:
        display(Image(filename=os.path.join(output_dir, chart_to_show)))
    except Exception as e:
        print(f"⚠️ Could not display image: {e}")

# Usage:
# show_chart()                # Shows the latest chart
# show_chart(image_name="your_chart.png")  # Shows a specific chart if present
show_chart()

---

# 🎯 Expected Output

When you have the correct tool packs, your chart should show:

| Region | Total Revenue |
|--------|---------------|
| EMEA | ~$12,994 |
| Americas | ~$9,499 |
| APAC | ~$1,499 |

**Did your chart match?** If not, think about:
- Do you have a tool to **get sales data**? (Which pack has query_sales?)
- Do you have a tool to **aggregate** the data for charting? (Which pack has aggregate_for_chart?)
- Do you have a tool to **create the chart**? (Which pack has create_chart?)

💡 **Hint:** The tools you need might be spread across different packs!

---

# 🔓 Solution (Don't peek until you've tried!)

<details>
<summary>Click to reveal the solution</summary>

The correct tool packs are:
1. **database** - For `query_sales` to get the sales data
2. **aggregator** - For `aggregate_for_chart` to transform raw data into chart format
3. **grapher** - For `create_chart` to create the visualization

**Why filesystem doesn't work:** The sales data is in the database, not in files!

**Why actions doesn't work:** Reports and notifications won't create charts!

**The key insight:** You need a **middleman tool** (`aggregate_for_chart`) to prepare the data. The agent can't go directly from `query_sales` to `create_chart` because `create_chart` expects aggregated data! And that middleman is in a **separate pack** from the chart creator!

```python
challenge_server.add_tool_pack("database")    # For query_sales
challenge_server.add_tool_pack("aggregator")  # For aggregate_for_chart  
challenge_server.add_tool_pack("grapher")     # For create_chart
```

</details>

---

# 🧹 Cleanup

When you're done, stop the MCP server:

In [ ]:
# Stop the challenge server
try:
    challenge_server.stop()
except:
    pass

print("✅ Server stopped. Workshop complete!")

---

# 📚 What You Learned

✅ **Traditional Agents** have coupled tools that are hardcoded

✅ **MCP Architecture** decouples tools from agents

✅ **MCPServerBuilder** makes it easy to create MCP servers

✅ **Tool Packs** provide pre-built functionality (filesystem, database, actions, grapher)

✅ **MCPToolsWrapper** connects agents to MCP servers

✅ **Tool selection matters** - choosing the right tools is key to solving problems!

---

## 🚀 Next Steps

1. **Explore the code**: Look at `lab.py` to see how the tools work
2. **Try different challenges**: Create other types of charts (pie, line)
3. **Understand the data flow**: Query → Transform → Visualize
4. **Build your own tools**: Modify `mcp_servers.py` to add custom tools

## 🔗 Resources

- [MCP Specification](https://modelcontextprotocol.io)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP Server Examples](https://github.com/modelcontextprotocol/servers)